In [434]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Dataset/Train.csv
/kaggle/input/Dataset/sample_submission.csv
/kaggle/input/Dataset/Test.csv


<h1 align="center"><u><b> Employee Attrition Rate Predictor</b></u></h1>
<h3 align="center"> Hackerearth machine learning challenge </h3>

### Problem Statement:
Employees are the most important part of an organization. Successful employees meet deadlines, make sales, and build the brand through positive customer interactions.

Employee attrition is a major cost to an organization and predicting such attritions is the most important requirement of the Human Resources department in many organizations. In this problem, your task is to predict the attrition rate of employees of an organization.

In [435]:
train_dataset_path = "/kaggle/input/Dataset/Train.csv"
test_dataset_path  = "/kaggle/input/Dataset/Test.csv"

In [436]:
# Data manipulation libraries
import pandas as pd
import numpy as np

# Data visualistaion libraries
import seaborn as sns
import matplotlib.pyplot as plt
pd.plotting.register_matplotlib_converters()
%matplotlib inline

# Machine learning libraries
from sklearn.metrics import mean_absolute_error as mae
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

In [437]:
# Both training and testing data
training_dataframe = pd.read_csv(train_dataset_path,index_col="Employee_ID")
testing_dataframe  = pd.read_csv(test_dataset_path,index_col ="Employee_ID")

### To gain a insight into the data

In [438]:
training_dataframe.describe()

,Age,Education_Level,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
count,6588.000000,7000.000000,6856.000000,7000.000000,7000.000000,7000.000000,7000.000000,6991.000000,6989.000000,7000.000000,6423.000000,7000.000000,6344.000000,7000.000000,7000.000000,7000.000000,7000.000000
mean,39.622799,3.187857,13.385064,2.367143,47.064286,0.817857,2.798000,6.006294,2.387895,3.098571,-0.008126,-0.013606,1.891078,2.834143,7.101286,3.257000,0.189376
std,13.606920,1.065102,10.364188,1.149395,15.761406,0.648205,1.163721,2.058435,1.122786,0.836377,0.989850,0.986933,0.529403,0.938945,1.164262,0.925319,0.185753
min,19.000000,1.000000,0.000000,0.000000,20.000000,0.000000,1.000000,1.000000,1.000000,1.000000,-1.817600,-2.776200,1.000000,1.000000,5.000000,1.000000,0.000000
25%,27.000000,3.000000,5.000000,1.000000,33.000000,0.000000,2.000000,5.000000,1.000000,3.000000,-0.961200,-0.453700,2.000000,2.000000,6.000000,3.000000,0.070400
50%,37.000000,3.000000,10.000000,2.000000,47.000000,1.000000,3.000000,6.000000,2.000000,3.000000,-0.104800,-0.453700,2.000000,3.000000,7.000000,3.000000,0.142650
75%,52.000000,4.000000,21.000000,3.000000,61.000000,1.000000,3.000000,8.000000,3.000000,3.000000,0.751600,0.707500,2.000000,3.000000,8.000000,4.000000,0.235000
max,65.000000,5.000000,43.000000,4.000000,74.000000,2.000000,5.000000,10.000000,5.000000,5.000000,1.608100,1.868800,3.000000,5.000000,9.000000,5.000000,0.995900


#### First and last few rows

In [439]:
training_dataframe.head()

,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,...,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
Employee_ID,,,,,,,,,,,,,,,,,,,,,
EID_23371,F,42.0,4,Married,Franklin,IT,Conceptual,4.0,4,33,...,type2,3.0,4,0.7516,1.8688,2.0,4,5,3,0.1841
EID_18000,M,24.0,3,Single,Springfield,Logistics,Analytical,5.0,4,36,...,type2,4.0,3,-0.9612,-0.4537,2.0,3,5,3,0.0670
EID_3891,F,58.0,3,Married,Clinton,Quality,Conceptual,27.0,3,51,...,type2,1.0,4,-0.9612,-0.4537,3.0,3,8,3,0.0851
EID_17492,F,26.0,3,Single,Lebanon,Human Resource Management,Behavioral,4.0,3,56,...,type2,1.0,3,-1.8176,-0.4537,NaN,3,7,3,0.0668
EID_22534,F,31.0,1,Married,Springfield,Logistics,Conceptual,5.0,4,62,...,type3,3.0,1,0.7516,-0.4537,2.0,2,8,2,0.1827


In [440]:
training_dataframe.tail()

,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,...,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
Employee_ID,,,,,,,,,,,,,,,,,,,,,
EID_16328,F,23.0,5,Married,Franklin,Operarions,Behavioral,4.0,1,20,...,type3,3.0,1,0.7516,-0.4537,2.0,3,7,3,0.7839
EID_8387,F,44.0,1,Married,Lebanon,R&D,Analytical,16.0,2,59,...,type3,3.0,2,-0.1048,-2.7762,2.0,4,8,4,0.1055
EID_8077,F,49.0,3,Single,Springfield,IT,Directive,10.0,1,59,...,type2,4.0,2,-0.1048,-0.4537,2.0,4,8,3,0.7847
EID_19597,F,47.0,3,Married,Washington,Sales,Behavioral,24.0,1,21,...,type0,2.0,4,0.7516,1.8688,2.0,3,6,3,0.4162
EID_1640,F,58.0,3,Married,Franklin,IT,Directive,22.0,2,74,...,type2,3.0,3,-0.1048,-1.6150,3.0,4,5,3,0.0910


## Preprocessing and Data Wrangling

To deal with columns having missing values we will perform imputation on the column with missing data. We might have dropped the column entirely but in that case the model might lost a lot of data, unless most values of the dropped column are missing.
We will first of all find out the colunms with missing data.

### Missing Values

In [441]:
## Columns with missing values
cols_with_missing = [col for col in training_dataframe.columns if training_dataframe[col].isnull().any()]
print(cols_with_missing)

['Age', 'Time_of_service', 'Pay_Scale', 'Work_Life_balance', 'VAR2', 'VAR4']


We will try to perform imputation on the missing values. But before imputation we will have to check for categorical values.

### Categorical Values
To deal with categorical values we can drop the column if not necessary, label encode the variables or perform ohe hot encoding.

In [442]:
# Get a list of categorical columns

s = (training_dataframe.dtypes == 'object')
object_cols = list(s[s].index)
print(object_cols)

['Gender', 'Relationship_Status', 'Hometown', 'Unit', 'Decision_skill_possess', 'Compensation_and_Benefits']


In [443]:
# Remove rows with missing target, separate target from predictors in training data
X_train_full = training_dataframe

X_train_full.dropna(axis=0, subset=['Attrition_rate'], inplace=True, how = "any")
y_train_full = X_train_full.Attrition_rate
X_train_full.drop(['Attrition_rate'], axis=1, inplace=True)


X_train_full.head()

,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,...,Pay_Scale,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7
Employee_ID,,,,,,,,,,,,,,,,,,,,,
EID_23371,F,42.0,4,Married,Franklin,IT,Conceptual,4.0,4,33,...,7.0,type2,3.0,4,0.7516,1.8688,2.0,4,5,3
EID_18000,M,24.0,3,Single,Springfield,Logistics,Analytical,5.0,4,36,...,6.0,type2,4.0,3,-0.9612,-0.4537,2.0,3,5,3
EID_3891,F,58.0,3,Married,Clinton,Quality,Conceptual,27.0,3,51,...,8.0,type2,1.0,4,-0.9612,-0.4537,3.0,3,8,3
EID_17492,F,26.0,3,Single,Lebanon,Human Resource Management,Behavioral,4.0,3,56,...,8.0,type2,1.0,3,-1.8176,-0.4537,NaN,3,7,3
EID_22534,F,31.0,1,Married,Springfield,Logistics,Conceptual,5.0,4,62,...,2.0,type3,3.0,1,0.7516,-0.4537,2.0,2,8,2


Now we will label encode the categorical columns. Fitting a label encoder to a column in the training data creates a corresponding integer-valued label for each unique value that appears in the training data. In case, the validation data contains values that don't also appear in the training data, the encoder will throw an error, because these values won't have an integer assigned to them.

In [444]:
X_valid_full = testing_dataframe

labelEncoder = LabelEncoder()
for col in object_cols:
    X_train_full[col] = labelEncoder.fit_transform(X_train_full[col])
    X_valid_full[col] = labelEncoder.transform(X_valid_full[col])
    

In [445]:
X_train_full.head()

,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,...,Pay_Scale,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7
Employee_ID,,,,,,,,,,,,,,,,,,,,,
EID_23371,0,42.0,4,0,1,2,2,4.0,4,33,...,7.0,2,3.0,4,0.7516,1.8688,2.0,4,5,3
EID_18000,1,24.0,3,1,3,3,0,5.0,4,36,...,6.0,2,4.0,3,-0.9612,-0.4537,2.0,3,5,3
EID_3891,0,58.0,3,0,0,8,2,27.0,3,51,...,8.0,2,1.0,4,-0.9612,-0.4537,3.0,3,8,3
EID_17492,0,26.0,3,1,2,1,1,4.0,3,56,...,8.0,2,1.0,3,-1.8176,-0.4537,NaN,3,7,3
EID_22534,0,31.0,1,0,3,3,2,5.0,4,62,...,2.0,3,3.0,1,0.7516,-0.4537,2.0,2,8,2


In [446]:
X_valid_full.head()

,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,...,Pay_Scale,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7
Employee_ID,,,,,,,,,,,,,,,,,,,,,
EID_22713,0,32.0,5,1,3,9,2,7.0,4,30,...,4.0,2,1.0,3,-0.9612,-0.4537,2.0,1,8,4
EID_9658,1,65.0,2,1,2,2,3,41.0,2,72,...,1.0,2,1.0,4,-0.9612,0.7075,1.0,2,8,2
EID_22203,1,52.0,3,0,3,10,3,21.0,3,25,...,8.0,3,1.0,4,-0.1048,0.7075,2.0,1,9,3
EID_7652,1,50.0,5,1,4,4,0,11.0,4,28,...,2.0,0,4.0,3,-0.1048,0.7075,2.0,2,8,3
EID_6516,0,44.0,3,0,1,9,2,12.0,4,47,...,2.0,2,4.0,4,1.6081,0.7075,2.0,2,7,4


## Imputation

In [447]:
si = SimpleImputer(strategy='median')
X_train_imputed = pd.DataFrame(si.fit_transform(X_train_full))
# It is important to fit_transform the training data


X_train_imputed.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0.0,42.0,4.0,0.0,1.0,2.0,2.0,4.0,4.0,33.0,...,7.0,2.0,3.0,4.0,0.7516,1.8688,2.0,4.0,5.0,3.0
1,1.0,24.0,3.0,1.0,3.0,3.0,0.0,5.0,4.0,36.0,...,6.0,2.0,4.0,3.0,-0.9612,-0.4537,2.0,3.0,5.0,3.0
2,0.0,58.0,3.0,0.0,0.0,8.0,2.0,27.0,3.0,51.0,...,8.0,2.0,1.0,4.0,-0.9612,-0.4537,3.0,3.0,8.0,3.0
3,0.0,26.0,3.0,1.0,2.0,1.0,1.0,4.0,3.0,56.0,...,8.0,2.0,1.0,3.0,-1.8176,-0.4537,2.0,3.0,7.0,3.0
4,0.0,31.0,1.0,0.0,3.0,3.0,2.0,5.0,4.0,62.0,...,2.0,3.0,3.0,1.0,0.7516,-0.4537,2.0,2.0,8.0,2.0


***Imputation removes column names so we wil have to put them back***

In [448]:
# Imputation removed column names; they have to be put back
X_train_imputed.columns = X_train_full.columns
# X_train_imputed.index = X_train_full.index
print(X_train_imputed.shape)

(7000, 22)


In [449]:
X_train_imputed.head()


,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,...,Pay_Scale,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7
0,0.0,42.0,4.0,0.0,1.0,2.0,2.0,4.0,4.0,33.0,...,7.0,2.0,3.0,4.0,0.7516,1.8688,2.0,4.0,5.0,3.0
1,1.0,24.0,3.0,1.0,3.0,3.0,0.0,5.0,4.0,36.0,...,6.0,2.0,4.0,3.0,-0.9612,-0.4537,2.0,3.0,5.0,3.0
2,0.0,58.0,3.0,0.0,0.0,8.0,2.0,27.0,3.0,51.0,...,8.0,2.0,1.0,4.0,-0.9612,-0.4537,3.0,3.0,8.0,3.0
3,0.0,26.0,3.0,1.0,2.0,1.0,1.0,4.0,3.0,56.0,...,8.0,2.0,1.0,3.0,-1.8176,-0.4537,2.0,3.0,7.0,3.0
4,0.0,31.0,1.0,0.0,3.0,3.0,2.0,5.0,4.0,62.0,...,2.0,3.0,3.0,1.0,0.7516,-0.4537,2.0,2.0,8.0,2.0
